In [1]:
from infrastructure.database.mongo_client import MongoDBClient
from infrastructure.repository import YoutubeContentRepository
from use_case import YouTubeGenerateTimelineSummary
from domain import YoutubeTimelineSummary

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

# MongoDB 클라이언트 초기화
client = MongoDBClient(uri=os.environ['MONGO_CONNECTION_STRING'])
client.connect()

# 저장소 초기화
repository = YoutubeContentRepository(client)

# LLM
openai_llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

# 유즈-케이스
youtube_generate_timeline_summary = YouTubeGenerateTimelineSummary(repository, openai_llm)

contents = repository.find_all()

count = len(contents)
for i, content in enumerate(contents):
    print(f'{i + 1}/{count}: {content.channel}: {content.title}: {content.url.url}')

    if content.timeline_summary is None:
        print("타임라인 요약 생성!!")
        youtube_generate_timeline_summary.execute(content.url.url)

KeyboardInterrupt: 